In [3]:
import numpy as np
def simplex_proj_rows(X, b=1.0):
    """
    각 행을 유클리드 심플렉스 { y >= 0, sum(y)=b }에 사영.
    Wang & Carreira-Perpiñán(2013) 알고리즘을 행 단위로 벡터화.

    Args:
        X (array_like): (N, d) 입력 행렬
        b (float): 심플렉스 반지름(기본 1.0, 양수)
    Returns:
        np.ndarray: (N, d) 사영 결과
    """
    X = np.asarray(X, dtype=float)
    if b <= 0:
        raise ValueError("b must be positive")
    N, d = X.shape
    if d == 0 or N == 0:
        return X.copy()

    # 1) 각 행을 내림차순 정렬
    U = np.sort(X, axis=1)[:, ::-1]         # (N, d)
    CSSV = np.cumsum(U, axis=1)             # (N, d)
    j = np.arange(1, d + 1)[None, :]        # (1, d)

    # 2) cond: u_j - (cssv_j - b)/(j) > 0  (여기서 j는 1..d)
    cond = U - (CSSV - b) / j > 0           # (N, d), True..True, False..False의 단조 패턴
    # 3) rho = 마지막으로 True인 위치
    #    cond가 단조이므로 True의 개수 - 1 이 마지막 True의 인덱스
    rho = cond.sum(axis=1) - 1              # (N,)

    # 4) theta = (sum_{i<=rho} u_i - b) / (rho + 1)
    rows = np.arange(N)
    theta = (CSSV[rows, rho] - b) / (rho + 1)  # (N,)

    # 5) y = max(x - theta, 0)
    Y = np.maximum(X - theta[:, None], 0.0)
    return Y


In [4]:
X = np.array([[0.2, -0.3, 2.0],
              [1.5, 1.5, -10.0],
              [-1.0, -1.0, -1.0]])
Y = simplex_proj_rows(X, b=1.0)
print(Y)
print(Y.sum(axis=1))     # 각 행이 1이어야 함
print((Y >= 0).all())    # 모든 성분 비음수

[[0.         0.         1.        ]
 [0.5        0.5        0.        ]
 [0.33333333 0.33333333 0.33333333]]
[1. 1. 1.]
True
